In [1]:
import numpy as np
import librosa
import soundfile as sf
from scipy.io import wavfile
from pedalboard import (
    Pedalboard,
    Convolution,
    Compressor,
    Chorus,
    Distortion,
    Gain,
    Reverb,
    Limiter,
    LadderFilter,
    Phaser,
)

In [2]:
fs = 44100

def save_wav(name, data):
    wavfile.write(name, fs, data.flatten().astype(np.float32))

# normalize data to loudest signal
def normalize(data):
    data_max = max(data)
    data_min = min(data)
    data_norm = max(data_max,abs(data_min))
    return data / data_norm

# add fadeout with length samples   
def apply_fadeout(audio, length):
    # convert to audio indices (samples)
    #length = int(duration*sr)
    end = len(audio)
    start = end - length
    # compute fade out curve
    # linear fade
    fade_curve = np.linspace(1.0, 0.0, length)
    audio[start:end] = audio[start:end] *fade_curve
    return audio

In [ ]:
# loads guitar signals, cuts them to 1 sec and rewrties them with fade out and normalized
# has been done already
'''
for input in range(1, 101, 1):
    x, samplerate = librosa.load('Trial_Data/GuitSigClean_Monophon/Monophon (%d).wav' %input, mono=True, sr=fs, duration=1.0, dtype=np.float32)          
    x = apply_fadeout(normalize(x),100)
    save_wav('Trial_Data/GuitSigClean_1/Monophon_1sec_(%d).wav' %(input), x)
    #with sf.SoundFile('Trial_Data/GuitSigClean_1/Monophon_1sec_(%d).wav' %(input), 'w', samplerate=fs, channels=len(x.shape)) as f:
    #    f.write(x)

x_test, ssmp = librosa.load('Trial_Data/GuitSigClean_1/Monophon_1sec_(1).wav', sr=44100, dtype=np.float32)
print(type(x_test))
'''

In [6]:
# writing pedalboard processed audio sorted by effect type
# reading processed audio and append to dry and wet array
from pathlib import Path
fx_list = [Chorus(), Phaser(), Distortion()]
fx_names = ['Chorus', 'Phaser', 'Distortion']
#dry = []
#wet = []
#effect_indices = []
directory_dry = 'Trial_Data//GuitSigClean_1'
files_dry = Path(directory_dry).glob('*')
for fx in range(0, 3, 1): # 0 = Chorus, 1 = Phaser, 2 = Distortion
    board = Pedalboard([fx_list[fx]], sample_rate=fs)
    input = 1
    for f in files_dry:
        x, samplerate = librosa.load(f, mono=True, sr=fs, duration=1.0, dtype=np.float32)
        x = apply_fadeout(normalize(x),100)
        output = board(x)
        output = apply_fadeout(output, 100)
        save_wav('Trial_Data/GuitSig_with_%s/Monophon_with_%s (%d).wav' %(fx_names[fx],fx_names[fx],input), output)
        input = input + 1

#dry = np.array(dry)
#wet = np.array(wet)
#effect_indices = np.array(effect_indices)
#dataset_dry = tf.data.Dataset.from_tensor_slices(dry)
#dataset_wet = tf.data.Dataset.from_tensor_slices(wet)
#dataset_effect_indices = tf.data.Dataset.from_tensor_slices(effect_indices)